In [1]:
using LinearAlgebra

In [2]:
function int_to_binarray(x::Int,y::Int)
    out = Array{Int8}(undef, y)
    t = x
    for i in 1:y
        out[i] = t % 2
        t ÷= 2
    end
    return out
end

int_to_binarray (generic function with 1 method)

In [3]:
int_to_binarray(32, 8)

8-element Array{Int8,1}:
 0
 0
 0
 0
 0
 1
 0
 0

In [4]:
struct Problem
    x1::Vector{Int8}
    x2::Vector{Int8}
    y::Vector{Int8}
    n::Int
end
function Problem(x1::Number, x2::Number, siz::Number)
    x1_in = Int64(x1)
    x2_in = Int64(x2)
    siz_in = Int64(siz)
    y_in = x1_in + x2_in
    return Problem(int_to_binarray(x1_in, siz_in), int_to_binarray(x2_in, siz_in), int_to_binarray(y_in, siz_in), siz)
end

function Problem(x1::Number, x2::Number)
    n = Int(ceil(log2(max(x1, x2))))
    return Problem(x1, x2, n)
end

Problem

In [5]:
t = Problem(162, 191)

Problem(Int8[0, 1, 0, 0, 0, 1, 0, 1], Int8[1, 1, 1, 1, 1, 1, 0, 1], Int8[1, 0, 0, 0, 0, 1, 1, 0], 8)

In [17]:
function calc_dims(n)
    return n == 1 ? 7*n+1 : 9*n-1
end

function σ(x::Real)
    return max(min(x, 1.0), 0.0)
end

function check_problem(p::Problem, m::Matrix)
    n = length(p.x1)
    ndims = calc_dims(n)
    v = 0.0*zeros(ndims)
    v[1] = 1.0
    v[2:n+1] = p.x1
    v[n+2:2*n+1] = p.x2 
    for i in 1:p.n+5
        v = σ.(m*v)
    end
    solution = v[2*n+2:3*n+1]
    out = 0
    for k in 1:length(solution)
        out += 2^(k-1)*solution[k]
    end
    return out
end

function define_solution(n)
    ndims = calc_dims(n)
    out = zeros((ndims, ndims))
    for i in 1:2*n+1
        out[i,i] = 1
    end
    #Xor(a, b) = or(a,b) and [not(a) or not(b)]
    s = 2*n+1 #Solution space goes from s+1 to s+n
    w = 3*n+1 #Workspace goes from w+1 to whatever
    f = 7*n + 1 #Final workspace for math with remainders
    xor_old = 0
    carry_old = 0

    for i in 1:n
        ws = w + 4*(i-1)
        ind1 = 1+i
        ind2 = 1+i+n

        #a or b
        out[ws+1, ind1] = 1
        out[ws+1, ind2] = 1

        #not(a) or not(b)
        out[ws+2, 1] = 2
        out[ws+2, ind1] = -1
        out[ws+2, ind2] = -1

        #Xor(a,b) -- result
        xor_ind = ws+3
        out[ws+3, 1] = -1
        out[ws+3, ws+1] = 1
        out[ws+3, ws+2] = 1

        #a and b -- carry
        carry_ind = ws+4
        out[ws+4, 1] = -1
        out[ws+4, ind1] = 1
        out[ws+4, ind2] = 1
        if carry_old > 0
            out[ws+4, carry_old] = 1
        end

        if i >= 2
            ws = f + 2*(i-2)
            out[ws+1, xor_ind] = 1
            out[ws+1, carry_old] = 1
            out[ws+2, 1] = 2
            out[ws+2, xor_ind] = -1
            out[ws+2, carry_old] = -1
            out[s+i, 1] = -1
            out[s+i, ws+1] = 1
            out[s+i, ws+2] = 1
        else
            out[s+i, ws+3] = 1
        end
        xor_old = xor_ind
        carry_old = carry_ind
    end
    return out
end

function check_problem(p::Problem)
    return check_problem(p, define_solution(p.n))
end
function check_problem(x1::Number, x2::Number)
    return check_problem(Problem(x1, x2))
end
function check_problem(x1::Number, x2::Number, n::Int)
    return check_problem(Problem(x1, x2, n))
end
check_problem(152,1057)

1209.0

Now we need to start constructing a neural network. The simple neural network will be a vector of nodes with a dense matrix of weights. This will probably turn into a sparse matrix for large numbers of nodes, but with a small number of nodes it's faster to do it as a dense matrix.

In [168]:
function increment_network(vec, mat, func, n_static)
    temp = mat * vec
    for i in 1:length(vec)
        if i <= n_static
            temp[i] = vec[i]
        else
            temp[i] = func(temp[i])
        end
    end
    return temp
end
σ = (x)->atan(x)*2/π

ErrorException: [91minvalid redefinition of constant σ[39m